In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import re

# 인재상 크롤링

In [2]:
MODEL_PATH = 'data/'
url = 'https://www.catch.co.kr/Comp/AnalysisCompGubun'

## 기업 분류 및 카테고리화

In [3]:
def category_classification(url):
    response = requests.get(url)
    categoryType = []
    categoriesDict = {}
    corpOfCategoryDict = {}
    bigCorps = {}

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        rawCorpData = soup.find("div",{"class": "look"})

        category1 = rawCorpData.find("p", {"id":"step1"}).get_text()  # 대기업
        category2 = rawCorpData.find("p", {"id":"step2"}).get_text()  # 산업별
        category3 = rawCorpData.find("p", {"id":"step3"}).get_text()  # 공공기관

        categoryType.append(category1)
        categoryType.append(category2)
        categoryType.append(category3)

        # 카테고리별 기업 리스트
        # [[대기업카테고리의 html], [산업카테고리의 html], [공공기관카테고리의 html], [공공기관카테고리의 html]]
        dataOfCategoryType = rawCorpData.find_all("div", {"class":"list"})

        # categoryType별 category 종류
        # [[삼성, sk][it미디어,  식품][]]
        categoriesList = [[categories.get_text().strip() for categories in dataOfCategoryType[i].find_all("p")] for i in range(len(dataOfCategoryType))]

        # index가 2인 카테고리 이름 지정 : "공공기관", 그리고 index 2, 3 합치기
        # 즉 [['공공기관'],['한전계열사', '금융공기업']] -> ['공공기관', '한전계열사', '금융공기업'] 으로 변환하는 작업
        categoriesList[2] = ["공공기관"]
        categoriesList[2].extend(categoriesList[3])
        del categoriesList[3]

        # categoriesDict -> (key: 카테고리의 타입, value: 카테고리 종류들)
        # (참고 - 대기업: 대기업회사종류, 업종별: 업종 종류, 공공기관: 공공기관종류)
        for i, type_ in enumerate(categoryType):
            categoriesDict[type_] = categoriesList[i]


        # 각 카테고리별 기업의 (기업이름, 기업분석 URL)
        subCategoriesList = [[[(corp.get_text(), corp.find("a")["href"]) for corp in corps.find_all("li")] for corps in dataOfCategoryType[i].find_all("ul")] for i in range(len(dataOfCategoryType))]

        # 공공기관 기업들을 하나로 묶어주고 계열사는 따로 분류
        # [['공공기관'],['한전계열사', '금융공기업']] -> ['공공기관', '한전계열사', '금융공기업'] 으로 변환에 맞추어 형태 변환
        # 계열사로 따로 분류한 기업들은 공공기관 기업안에 이미 들어가 있음 -> 인재상 찾을 때는 공공기관에 있는 기관들만 검색
        temp = []
        for i in subCategoriesList[2]:
            temp.extend(i)
        subCategoriesList[2] = [temp]

        for i in subCategoriesList[3]:
            subCategoriesList[2].append(i)
        del subCategoriesList[3]

        for i, categories in enumerate(categoriesList):
            for j, category in enumerate(categories):
                corpOfCategoryDict[category] = subCategoriesList[i][j]

    else : 
        print(response.status_code)
    
    return categoryType, categoriesDict, corpOfCategoryDict

In [4]:
categoryType, categoriesDict, corpOfCategoryDict = category_classification(url)
categoryType
categoriesDict
corpOfCategoryDict

{'삼성': [('삼성디스플레이', '/Comp/AnalysisCompView?ID=975'),
  ('삼성물산', '/Comp/AnalysisCompView?ID=1151'),
  ('삼성생명보험', '/Comp/AnalysisCompView?ID=891'),
  ('삼성전기', '/Comp/AnalysisCompView?ID=977'),
  ('삼성전자', '/Comp/AnalysisCompView?ID=1065'),
  ('삼성전자로지텍', '/Comp/AnalysisCompView?ID=978'),
  ('삼성중공업', '/Comp/AnalysisCompView?ID=515'),
  ('삼성증권', '/Comp/AnalysisCompView?ID=1113'),
  ('삼성카드', '/Comp/AnalysisCompView?ID=613'),
  ('삼성화재해상보험', '/Comp/AnalysisCompView?ID=614'),
  ('삼성SDS', '/Comp/AnalysisCompView?ID=974'),
  ('제일기획', '/Comp/AnalysisCompView?ID=1068'),
  ('호텔신라', '/Comp/AnalysisCompView?ID=1174')],
 'GS': [('GS건설', '/Comp/AnalysisCompView?ID=840'),
  ('GS리테일', '/Comp/AnalysisCompView?ID=853'),
  ('GS칼텍스', '/Comp/AnalysisCompView?ID=1083'),
  ('GS홈쇼핑', '/Comp/AnalysisCompView?ID=1086')],
 'LG': [('LG CNS', '/Comp/AnalysisCompView?ID=1067'),
  ('LG디스플레이', '/Comp/AnalysisCompView?ID=1150'),
  ('LG생활건강', '/Comp/AnalysisCompView?ID=859'),
  ('LG유플러스', '/Comp/AnalysisCompView?ID=860'),


In [5]:
corpsInfo = [corp for corps in list(corpOfCategoryDict.values()) for corp in corps]
# 중복기업 제거
print('중복기업 제거:', len(corpsInfo), '->',len(set(corpsInfo)))
corpsInfo = list(set(corpsInfo))
corpsInfo

중복기업 제거: 546 -> 453


[('동부건설', '/Comp/AnalysisCompView?ID=1120'),
 ('SK에코플랜트', '/Comp/AnalysisCompView?ID=1109'),
 ('메리츠화재해상보험', '/Comp/AnalysisCompView?ID=336'),
 ('교보문고', '/Comp/AnalysisCompView?ID=1140'),
 ('HDC현대산업개발', '/Comp/AnalysisCompView?ID=729'),
 ('넥센타이어', '/Comp/AnalysisCompView?ID=88'),
 ('JW중외제약', '/Comp/AnalysisCompView?ID=1138'),
 ('GS칼텍스', '/Comp/AnalysisCompView?ID=1083'),
 ('에스엘', '/Comp/AnalysisCompView?ID=901'),
 ('포스코ICT', '/Comp/AnalysisCompView?ID=1108'),
 ('코스콤', '/Comp/AnalysisCompView?ID=800'),
 ('아시아나항공', '/Comp/AnalysisCompView?ID=983'),
 ('SK플래닛', '/Comp/AnalysisCompView?ID=1092'),
 ('롯데정밀화학', '/Comp/AnalysisCompView?ID=969'),
 ('롯데캐피탈', '/Comp/AnalysisCompView?ID=971'),
 ('현대로템', '/Comp/AnalysisCompView?ID=929'),
 ('대웅제약', '/Comp/AnalysisCompView?ID=1134'),
 ('건강보험심사평가원', '/Comp/AnalysisCompView?ID=1004'),
 ('진에어', '/Comp/AnalysisCompView?ID=988'),
 ('코카콜라음료', '/Comp/AnalysisCompView?ID=1125'),
 ('유한킴벌리', '/Comp/AnalysisCompView?ID=908'),
 ('오비맥주', '/Comp/AnalysisCompView?ID=

## 기업별 인재상 데이터 추출

**get_data()함수에서 사용된 case 종류**
* 2개의 타입의 데이터가 있음 1. 테이블, 2. 동그라미 표
* 1. 테이블 타입
    * 제목이 인재상이고 소제목이 인재상 내용 (테이블자체가 인재상) - **case 1**
    * 소제목에 인재상이 있고 소제목 오른쪽 내용에 인재상 내용이 적힌경우 
        * 인재상 내용이 dt dd 속성으로 이루어진 경우 - **case 2**
        * p 속성으로 이루어진 경우 - (class:kind) - **case 3**
* 2. 동그라미 표 타입
    * 제목에 인재상이 적혀있는 경우 - **case 4**
    * 제목에 인재상이 없지만 tag로 인재상이 적혀 있는 경우 - **case 5**
    * 모두 div(class:subj) dd로 이루어짐

In [6]:
url2 = "https://www.catch.co.kr/Comp/AnalysisCompView?ID=975"
p = re.compile("#[^#]*")

# 기업분석의 기업 추구 가치 부분에서 사용된 주제 추출
def check_data(name, url):
    response = requests.get(url)
    question1 = []  # 큰 주제의 제목 (h4)
    question2 = []  # 소주제의 제목
    tagList = []
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        wantPeople = soup.find("div",{"id": "step41"})
        
        try:
            corpAnalysis = wantPeople.find_all("div", {"class":"anal_ht"})
            
        except:
            print("기업 문화는 어떨까? 부분이 없음", name)
            return name, None, None, -1

        for analysis in corpAnalysis:
            # 큰 주제 (h4에 해당)
            subject4 = analysis.find("h4")
            main_subject = p.sub('', subject4.get_text())
            tags = p.findall(subject4.get_text())
            
            question1.append(main_subject)
            tagList.extend([tag.strip() for tag in tags])


            # 작은 주제 TypeA처럼 table형식에서의 소주제
            table = analysis.find("tbody")

            try:
                contents = table.find_all("tr")
            except:
                contents = ""

            for content in contents:
                subSubject = content.find("span", {"class":"txt"})
                question2.append(subSubject.get_text())
                   
    else : 
        print(response.status_code)
    
    return question1, question2, tagList, 1

# 기업분석의 기업 추구 가치부분에서 특정 주제에 대한 내용 추출 - "인재"
def get_data(name, url, questions, tagList):
    response = requests.get(url)
    wantPeopleList = []
    wantPeopleList_b = []
    infoInTable = False    
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        wantPeople = soup.find("div",{"id": "step41"})
        
        try:
            corpAnalysis = wantPeople.find_all("div", {"class":"anal_ht"})
            
        except:
            print("기업 문화는 어떨까? 부분이 없음", name)
            return name, None, -1
        
        
        for analysis in corpAnalysis:
            tableFlag = False
            tagFlag = False
            
            # 작은 주제 TypeA처럼 table형식에서의 소주제
            table = analysis.find("tbody")
            if table is not None:
                tableFlag = True
                contents = table.find_all("tr")

                for content in contents:
                    subSubject = content.find("span", {"class":"txt"})

                    if subSubject.get_text() in questions:
                        infoInTable = True
                        TypeB_keyword = content.find_all("dt")
                        keywordB_describe = content.find_all("dd")
                        
                        # ********************* case 3 *********************
                        if len(TypeB_keyword) == 0 or len(keywordB_describe) == 0:
                            kind = content.find("div", {"class":"kind"})
                            if kind is not None:
                                kindList = kind.find_all("p")
                                for k in kindList:
                                    wantPeopleList_b.append(k.get_text())
                            else:
                                print("다른 경우 <1>({}) - {}".format(name, url))
                                
                        # ********************* case 2 *********************
                        for key, value in zip(TypeB_keyword, keywordB_describe):
                            wantPeopleList.append((key.get_text(), value.get_text()))
            
            
            # 큰 주제 (h4에 해당)
            subject4 = analysis.find("h4")
            main_subject = p.sub('', subject4.get_text())
            tags = p.findall(subject4.get_text())
            
            for tag in [tag.strip() for tag in tags]:
                if tag in tagList: 
                    tagFlag = True
                    
            if main_subject in questions:
                # ********************* case 1 *********************
                if tableFlag == True:
                    for content in contents:
                        kindList = content.find_all("div", {"class":"kind"})
                        for k in kindList:
                            attrPs = k.find_all("p")
                            for attrP in attrPs:
                                wantPeopleList_b.append(attrP.get_text())
                # ********************* case 4 *********************
                else:
                    TypeA_keyword = analysis.find_all("div", {"class":"subj"})
                    keywordA_describe = analysis.find_all("dd")
                    if len(TypeA_keyword) == 0 or len(keywordA_describe) == 0:
                        print("다른 경우 <2>({}) - {}".format(name, url))
                        
                    for key, value in zip(TypeA_keyword, keywordA_describe):
                            wantPeopleList.append((key.get_text(), value.get_text()))
            # ********************* case 5 *********************                
            elif tagFlag:
                if tableFlag != True:
                    TypeA_keyword = analysis.find_all("div", {"class":"subj"})
                    keywordA_describe = analysis.find_all("dd")
                    if len(TypeA_keyword) == 0 or len(keywordA_describe) == 0:
                        print("다른 경우 <3>({}) - {}".format(name, url))
                        
                    for key, value in zip(TypeA_keyword, keywordA_describe):
                            wantPeopleList.append((key.get_text(), value.get_text()))
                
    else : 
        print(response.status_code)
    
    return wantPeopleList, wantPeopleList_b, 1

In [7]:
noDataCorp = []
questions1 = []
questions2 = []
tagList = []

for name, url in corpsInfo:
    url = "https://www.catch.co.kr" + url
    q1, q2, tags, state = check_data(name, url)
    if state == -1:
        # 기업분석이 없는 기업이름 저장
        noDataCorp.append(q1)
    else:
        questions1.extend(q1)
        questions2.extend(q2)
        tagList.extend(tags)

기업 문화는 어떨까? 부분이 없음 메리츠화재해상보험
기업 문화는 어떨까? 부분이 없음 현대로템
기업 문화는 어떨까? 부분이 없음 한국산업기술진흥원
기업 문화는 어떨까? 부분이 없음 하나금융투자
기업 문화는 어떨까? 부분이 없음 한국쓰리엠
기업 문화는 어떨까? 부분이 없음 그랜드코리아레저
기업 문화는 어떨까? 부분이 없음 한전KPS
기업 문화는 어떨까? 부분이 없음 삼성중공업
기업 문화는 어떨까? 부분이 없음 한국자산관리공사
기업 문화는 어떨까? 부분이 없음 신한금융투자
기업 문화는 어떨까? 부분이 없음 한화에어로스페이스
기업 문화는 어떨까? 부분이 없음 동원F&B;
기업 문화는 어떨까? 부분이 없음 스마트스터디
기업 문화는 어떨까? 부분이 없음 도로교통공단
기업 문화는 어떨까? 부분이 없음 한국지역난방공사
기업 문화는 어떨까? 부분이 없음 비테스코테크놀로지스코리아
기업 문화는 어떨까? 부분이 없음 미래에셋증권
기업 문화는 어떨까? 부분이 없음 인천교통공사
기업 문화는 어떨까? 부분이 없음 S-OIL
기업 문화는 어떨까? 부분이 없음 한국장학재단
기업 문화는 어떨까? 부분이 없음 한국투자증권
기업 문화는 어떨까? 부분이 없음 LX하우시스
기업 문화는 어떨까? 부분이 없음 코레일유통
기업 문화는 어떨까? 부분이 없음 한화건설
기업 문화는 어떨까? 부분이 없음 롯데케미칼
기업 문화는 어떨까? 부분이 없음 한국승강기안전공단
기업 문화는 어떨까? 부분이 없음 광주은행
기업 문화는 어떨까? 부분이 없음 한국가스안전공사
기업 문화는 어떨까? 부분이 없음 에릭슨엘지
기업 문화는 어떨까? 부분이 없음 동서식품
기업 문화는 어떨까? 부분이 없음 하이투자증권
기업 문화는 어떨까? 부분이 없음 한국산업은행
기업 문화는 어떨까? 부분이 없음 한국도로공사
기업 문화는 어떨까? 부분이 없음 네오위즈
기업 문화는 어떨까? 부분이 없음 팜한농
기업 문화는 어떨까? 부분이 없음 한국가스기술공사
기업 문화는 어떨까? 부분이 없음 정보통신산업진흥원
기업 문화는 어떨까? 부분이 없음 삼성SDI
기

* 인재상 특이 문항
    * 삼천리 : 어떤 인재를 원할까?'사랑'을 품은 열정인
    * 코카콜라음료 : we want THIS

In [8]:
# 기업 분석이 있는 기업들만 저장
yesDataCorp = []
for corp in corpsInfo:
    if not (corp[0] in noDataCorp):
        yesDataCorp.append(corp)

In [9]:
questions1 = list(set(questions1))
questions2 = list(set(questions2))
tagList = list(set(tagList))
print("큰주제의 종류 개수: ", len(questions1))
print("소주제의 종류 개수: ", len(questions2))
print("tag의 종류 개수: ", len(questions2))

큰주제의 종류 개수:  29
소주제의 종류 개수:  174
tag의 종류 개수:  174


In [10]:
wantQ1 = [i for i in questions1 if ('인재' in i) or ('want' in i)]
wantQ2 = [i for i in questions2 if '인재' in i]
wantTag = [i for i in tagList if '인재' in i]
print(wantQ1)
print(wantQ2)
print(wantTag)

['어떤 인재를 원할까?', "어떤 인재를 원할까?'사랑'을 품은 열정인", 'We want THIS', '인재상']
['인재상', '어떤 인재를 원할까?']
['#인재상', '#신통한인재', '#현대백화점그룹 인재상', '#상사부문_인재상', '#배려와 존중으로 상대방을 먼저 생각하는 창조적인 전문가형 인재']


In [11]:
wantQuestions = wantQ1 + wantQ2
wantQuestions = list(set(wantQuestions))
wantQuestions

["어떤 인재를 원할까?'사랑'을 품은 열정인", '인재상', 'We want THIS', '어떤 인재를 원할까?']

In [12]:
wantPeopleList = []
wantPeopleList_b = []
wantPeople_dic = {}
wantPeople_b_dic = {}

for name, url in yesDataCorp:
    data, data_b = [], []
    url = "https://www.catch.co.kr" + url
    data, data_b, state = get_data(name, url, wantQuestions, wantTag)
    if state == -1:
        # 기업분석이 없는 기업이름 저장
        noDataCorp.append(data)
    else:
        #### data_b를 처리해야 됨
        wantPeopleList.extend(data)
        wantPeopleList_b.extend(data_b)
        if len(data_b) != 0:
            wantPeople_b_dic[name] = data_b
            
        wantPeople_dic[name] = data

### 이상한 데이터 처리
* p속성으로 추출한 데이터

In [13]:
# 이상한 데이터 확인
for name, value in wantPeople_b_dic.items():
    print(name)
    print(value)
    print()

JW중외제약
['Hi-Focuser', '생명존중과 개척정신을 계승한 Hi-Focuser는 환자 중심의 사고를 바탕으로 변화, 성장, 도전에 Focus하여 우리의 독보적 가치와 존재의 이유를 높이는 JW그룹의 인재상입니다.']

SK플래닛
['', '1. 포장하지 말고, 본질가치에 집중하자.2. 현장에서 답을 찾고 실행한다.3. 주인의식으로 업무에 임한다.4. 고객보다 앞서 생각하고 제안한다.5. 기탄없이 비판하고 토론한다.']

삼성바이오로직스
['바이오 인재상', '회사 비전과 목표를 공유하며, 강한 열정과 책임감으로 해당분야 최고의 전문성과 실행력을 통해 성과창출을 극대화하고, 협업으로 집단지성을 추구하는 T자형 글로벌 인재']

셀트리온
['', '· 창의적인 사고와 발상의 전환을 통해 새로운 아이디어를 도출하고 문제를 해결하는 인재를 원합니다.· 생명을 중시하며 엄정한 책임의식을 바탕으로 원칙을 준수하는 인재를 원합니다.· 불굴의 도전정신을 가지고 업무에 임하며, 변화와 실패에 대한 두려움이 없는 인재를 원합니다.· 세계 시장을 무대로 경쟁하고, 셀트리온의 Vision을 실현할 수 있는 열정이 있는 인재를 원합니다.']

두산
['세계 속의 자랑스러운 두산두산 Credo의 아홉 가지 핵심 가치', '자랑스러운 두산이란, 임직원을 포함한 모든 이해관계자들이 두산을 통해 자부심과 자랑스러움을 느낀다는 의미입니다.']

삼성증권
['7 Pinciples : 성장과 행복을 위한 우리의 약속', '1. 우리는 형식보다 내용을 더 중요하게 생각합니다.2. 우리는 작은 것부터 변화를 시도합니다.3. 우리는 스스로 질문하며 더 나음을 고민합니다.4. 우리는 무엇이든 제대로 하기 위해 노력합니다.5. 우리는 권위의식 없이 소통하며 서로를 배려합니다.6. 우리는 안되는 이유 대신 되는 방법을 함께 찾습니다.7. 우리는 가족에게 부끄러운 일은 하지 않습니다.']

농심
['', '1. 충직農心은 天心충성스럽고 정직하게 자신의 업무를 수행하는 사람2. 도전最高로 큰 目標를

In [14]:
# 이상한 데이터 수작업으로 진행
wantPeople_b_dic['LG화학'] = [("", "꿈과 열정을 가지고 세계 최고에 도전하는 사람"), 
                    ("", "팀웍을 이루며 자율적이고 창의적으로 일하는 사람"),
                    ("", "고객을 최우선으로 생각하고 끊임없이 혁신하는 사람"),
                    ("", "꾸준히 실력을 배양하여 정정당당하게 경쟁하는 사람")]

wantPeople_b_dic['셀트리온'] = [("", "창의적인 사고와 발상의 전환을 통해 새로운 아이디어를 도출하고 문제를 해결하는 인재"), 
                        ("", "생명을 중시하며 엄정한 책임의식을 바탕으로 원칙을 준수하는 인재"),
                        ("", "불굴의 도전정신을 가지고 업무에 임하며, 변화와 실패에 대한 두려움이 없는 인재"),
                        ("", "세계 시장을 무대로 경쟁하고, 셀트리온의 Vision을 실현할 수 있는 열정이 있는 인재")]

wantPeople_b_dic['농심'] = [('충직','충성스럽고 정직하게 자신의 업무를 수행하는 사람'),
                    ('도전','실패를 두려워하지 않고 스스로를 극복하여 해내는 사람'),
                    ('창조','유연한 사고를 숙고하여 근본부터 뒤집어 볼 수 있는 사람'),
                    ('업무 전문가','맡은 분야의 업무지식과 핵심역량을 갖추어 최고를 지향하는 사람'),
                    ('사업 전문가','시장 통찰력과 올바른 전략으로 사업을 성공적으로 추진하는 사람'),
                    ('Global 전문가','글로벌 업무 경쟁력을 바탕으로 글로벌 시장을 개척해내는 사람')]

wantPeople_b_dic['삼성바이오로직스'] = [('바이오 인재상','회사 비전과 목표를 공유하며, 강한 열정과 책임감으로 해당분야 최고의 전문성과 실행력을 통해 성과창출을 극대화하고, 협업으로 집단지성을 추구하는 T자형 글로벌 인재')]

wantPeople_b_dic['두산'] = [('','세계 속의 자랑스러운 두산두산 Credo의 아홉 가지 핵심 가치'),
                    ('','자랑스러운 두산이란, 임직원을 포함한 모든 이해관계자들이 두산을 통해 자부심과 자랑스러움을 느낀다는 의미입니다.')]

wantPeople_b_dic['JW중외제약'] = [('Hi-Focuser', '생명존중과 개척정신을 계승한 Hi-Focuser는 환자 중심의 사고를 바탕으로 변화, 성장, 도전에 Focus하여 우리의 독보적 가치와 존재의 이유를 높이는 JW그룹의 인재상입니다.')]

wantPeople_b_dic['SK플래닛'] = [("", "포장하지 말고, 본질가치에 집중"), 
                        ("", "현장에서 답을 찾고 실행"),
                        ("", "주인의식으로 업무에 임한다"),
                        ("", "고객보다 앞서 생각하고 제안한다"),
                        ("", "기탄없이 비판하고 토론한다")]

wantPeople_b_dic['LS일렉트릭'] = [('Great Company', '고객에게 사랑받는 최고의 회사가 되어')]

wantPeople_b_dic['이수화학'] = [("", "기본적인 생활과 업무수행에 있어서 조직의 핵심가치가 제시하는 원칙을 바탕으로 행동하는 이수인"), 
                        ("", "창의적이고 능동적인 이수인"),
                        ("", "현재에 안주하지 않고, 늘 새로운 것에 대한 고민으로 창의적 열정이 가득한 이수인"),
                        ("", "새로운 것을 적극적으로 제안하고 실행할 수 있는 추진력을 보유한 이수인"),
                        ("", "언제나 성장하고 변화하는 이수인"),
                        ("", "스스로의 성장하고자 하는 노력을 통해 개인과 조직의 발전에 기여하는 이수인"),
                        ("", "개인의 혁신과 변화를 통해 조직의 변화를 주도하는 이수인")]

wantPeople_b_dic['우리은행'] = [("", "올바른 품성을 가진, 도전적이고 창의적인 최고의 금융전문가")]

wantPeople_b_dic['디엘'] = [("", "새로운 것을 찾는다"), 
                    ("", "자기 일에 으뜸이 된다"),
                    ("", "팀웍을 이룬다"),
                    ("", "멀리 내려다본다"),
                    ("", "고객을 잘안다"),
                    ("", "약속을 지킨다"),
                    ("", "근검절약을 한다")]

wantPeople_b_dic['삼성증권'] = [("", "우리는 형식보다 내용을 더 중요하게 생각합니다"), 
                    ("", "우리는 작은 것부터 변화를 시도합니다"),
                    ("", "우리는 스스로 질문하며 더 나음을 고민합니다"),
                    ("", "우리는 무엇이든 제대로 하기 위해 노력합니다"),
                    ("", "우리는 권위의식 없이 소통하며 서로를 배려합니다"),
                    ("", "우리는 안되는 이유 대신 되는 방법을 함께 찾습니다"),
                    ("", "우리는 가족에게 부끄러운 일은 하지 않습니다")]

wantPeople_b_dic['OCI'] = [("OUR VALUES", "도전정신을 가지고 철저한 준비를 통해 기회를 포착하여 더 나은 미래를 위해 끊임없이 변화")]

In [15]:
for v in wantPeople_b_dic.values():
    wantPeopleList.extend(v)

for name, values in wantPeople_b_dic.items():
    wantPeople_dic[name].extend(values)
    print(wantPeople_dic[name])
    print()

[('Hi-Focuser', '생명존중과 개척정신을 계승한 Hi-Focuser는 환자 중심의 사고를 바탕으로 변화, 성장, 도전에 Focus하여 우리의 독보적 가치와 존재의 이유를 높이는 JW그룹의 인재상입니다.')]

[('', '포장하지 말고, 본질가치에 집중'), ('', '현장에서 답을 찾고 실행'), ('', '주인의식으로 업무에 임한다'), ('', '고객보다 앞서 생각하고 제안한다'), ('', '기탄없이 비판하고 토론한다')]

[('바이오 인재상', '회사 비전과 목표를 공유하며, 강한 열정과 책임감으로 해당분야 최고의 전문성과 실행력을 통해 성과창출을 극대화하고, 협업으로 집단지성을 추구하는 T자형 글로벌 인재')]

[('', '창의적인 사고와 발상의 전환을 통해 새로운 아이디어를 도출하고 문제를 해결하는 인재'), ('', '생명을 중시하며 엄정한 책임의식을 바탕으로 원칙을 준수하는 인재'), ('', '불굴의 도전정신을 가지고 업무에 임하며, 변화와 실패에 대한 두려움이 없는 인재'), ('', '세계 시장을 무대로 경쟁하고, 셀트리온의 Vision을 실현할 수 있는 열정이 있는 인재')]

[('육성인화', '두산인은 사람을 성과의 도구로 생각하지 않으며, 하나의 인격체로 존중합니다.두산인은 조직 공동의 하나된 목표달성을 위해 자발적으로 합심하는 팀워크를 중시하고 실천합니다'), ('눈높이소통', '두산인은 무조건 1등만을 목표로 삼기보다 성취하였을 때 스스로 큰 자부심을 가질 만한 목표를 세웁니다.두산인은 언제든지 자신 있게 자신의 생각과 합리적 근거를 표현하고 다른 사람의 의견을 경청합니다.'), ('근성집중', '두산인은 문제의 해법을 찾을 때까지 포기하지 않고 모든 종류의 가능성을 시도하여 돌파구를 찾습니다.두산인은 가장 중요한 것을 찾아내고 이를 실행하는 데 집중합니다. 중요한 것을 찾아내면, 모든 가용자원을 총동원하여 이를 확실하게 먼저 해결해냅니다.'), ('', '세계 속의 자랑스러운 두산두산 Credo의 아홉 가지 핵심

In [16]:
wantPeople_dic['OCI']

[('CHANCE',
  'Seize Emerging Opportunities잠재적인 기회를 찾아내기 위해 관심을 보이고, 새로운 기회를 포착하면 신속히 결단하여 경쟁력 확보에 힘쓴다.Faster Than Market Change빠르게 변화하는 시장에 대한 정보를 신속히 파악 하여 시의 적절히 대응한다.'),
 ('CHALLENGE',
  'Innovative Solution도전과 실패를 두려워하지 않으며, 문제 처리에 있어 관행적인 방식보다는 새로운 방식을 선호 하고 능동적으로 행동한다.Entrepreneurial Thinking지속가능한 가치를 추구하고, 고객의 니즈 충족과 회사 이익에 부합되는 대안을 효과적으로 제시한다.'),
 ('CHANGE',
  'Global Outlook국제동향에 주의를 기울이며, 외부시장 변화에 주도적으로 대처한다.Flexible Open Mind다양한 문화와 사고방식에 대해 이해하고 공감 하는 자세를 가진다.'),
 ('OUR VALUES', '도전정신을 가지고 철저한 준비를 통해 기회를 포착하여 더 나은 미래를 위해 끊임없이 변화')]

In [17]:
print(len(wantPeopleList))
print(len(wantPeople_dic.keys()))

1127
377


**유형**
* ("키워드", "설명")
* ("", "설명")
* ("키워드", "")
* ("큰범주", "설명(키워드, 설명)") - OCI

### 없는 데이터 처리
* 인재상에 대한 데이터가 없는 경우
* 인재상의 정보가 핵심 가치에 적혀있는 경우

In [18]:
for name, values in wantPeople_dic.items():
    if len(values) == 0:
        print(name)

교보문고
HDC현대산업개발
에스엘
롯데정밀화학
진에어
오비맥주
무신사
한국수자원공사
한국인터넷진흥원
CJ ENM
데상트코리아
KCC
대교
SK에너지
국민연금공단
신한카드
서울주택도시공사
대우건설
CJ CGV
한국씨티은행
KB증권
LG CNS
LS전선
마이다스아이티
한국수출입은행
금호타이어
신영증권
DB생명보험
에이치케이이노엔
에어서울
삼성엔지니어링
KT
현대차증권
국립생태원
CJ대한통운
한국환경정책평가연구원
유진투자증권
라인프렌즈
오렌지라이프생명보험
딜리버리히어로코리아
웅진씽크빅
카카오뱅크
코레일테크
한국방송공사
한국과학기술원
현대모비스
동우화인켐
티웨이항공
한국원자력안전기술원
엔씨소프트
비바리퍼블리카
NHN
국토연구원
금호건설
한국조폐공사
한양
국민건강보험공단
근로복지공단
악사손해보험
현대백화점
현대카드
우리카드
현대글로비스
대한항공
두산인프라코어
한국가스공사
이랜드월드
야놀자
라인플러스
한국문화정보원
나스미디어
한국아이비엠
SBS
카카오게임즈
현대오일뱅크
NH투자증권
hy
CJ제일제당
오뚜기
매일유업
한국콜마
네이버
이노션
이마트
한미약품
BGF리테일
미래에셋자산운용
현대자동차
한국전력공사
LG디스플레이
신세계인터내셔날
에스엠엔터테인먼트
코웨이
쿠팡


### 카테고리 분류

#### 확인한 것 분류

In [19]:
def categorize(keyword, data_li):
    group = []
    
    for f in data_li[:]:
        if keyword in f[0]:
            data_li.pop(data_li.index(f))
            group.append(f)
        
    print('남은 데이터 개수: ', len(data_li))
    
    return group

In [20]:
wantPeople_copy = wantPeopleList[:]
wantCategoryDic = {}
keywords = ['혁신', '도전', '창의', '전문', '신뢰', '변화', '고객', '팀워크', '팀웍', '책임', '소통', '성실', '열심']

for w in keywords:
    wantCategoryDic[w] = categorize(w, wantPeople_copy)

남은 데이터 개수:  1071
남은 데이터 개수:  994
남은 데이터 개수:  955
남은 데이터 개수:  876
남은 데이터 개수:  838
남은 데이터 개수:  825
남은 데이터 개수:  788
남은 데이터 개수:  779
남은 데이터 개수:  777
남은 데이터 개수:  757
남은 데이터 개수:  723
남은 데이터 개수:  718
남은 데이터 개수:  717


In [21]:
import pickle

f = open("wantPeople_dic.pickle", "wb")
pickle.dump(wantPeople_dic, f)
f.close()
f = open("wantPeopleList.pickle", "wb")
pickle.dump(wantPeopleList, f)
f.close()

In [3]:
import pickle
f = open("wantPeopleList.pickle", "rb")
wantPeopleList = pickle.load(f)
f.close()

f = open("wantPeople_dic.pickle", "rb")
wantPeople_dic = pickle.load(f)
f.close()

In [4]:
wantPeople_dic

{'동부건설': [('Basic', '기본에 충실한 인재동부건설의 3가지 기본원칙 품질, 준법, 안전을 실천하는 사람'),
  ('Ownership',
   '주인의식을 가진 인재능동적인 자세로 책임감 있게 업무에 임하며 자신의 성장과 회사의 발전을 함께 추구하는 사람'),
  ('Passion',
   '열정적으로 도전하는 인재현실에 안주하지 않고 배움의 자세로 항상 더 높은 목표를 향해 끊임없이 노력하는 열정을 가진 사람'),
  ('Creative',
   '창의적 사고를 가진 인재참신한 관점으로 문제를 바라보고 창의적인 해결방안을 도출하여 새로운 가치를 창출하는 사람'),
  ('Respect',
   '다양성을 존중하는 인재서로의 차이를 존중하며 신뢰관계를 형성하고, 솔선수범하는 자세로 다양한 이해관계자들과 시너지를 이끌어내는 사람')],
 'SK에코플랜트': [('따뜻한', '자긍심공동체의식상호배려'),
  ('프로페셔널', '내일 알기도전적 목표 설정철저하고 즐거운 실행지식 공유')],
 '교보문고': [],
 'HDC현대산업개발': [],
 '넥센타이어': [('도전정신', '최고의 전문성을 기반으로 실패를 두려워하지 않고 자율적으로 변화를 주도하는 넥센타이어人'),
  ('창의성', '창의적인 사고방식을 기반으로 결과에 안주하지 않고 미래를 예측하여 최선의 성과를 창출하기 위해 노력하는 넥센타이어人'),
  ('협력', '신뢰를 기반으로 적극적이고 긍정적인 마인드를 통해 협력을 극대화하는 넥센타이어人'),
  ('글로벌마인드',
   '통찰력을 기반으로 고객을 위한 전략적 선택과 집중을 통해 Global Market Leader로서의 역할을 수행하는 넥센타이어人')],
 'JW중외제약': [('Hi-Focuser',
   '생명존중과 개척정신을 계승한 Hi-Focuser는 환자 중심의 사고를 바탕으로 변화, 성장, 도전에 Focus하여 우리의 독보적 가치와 존재의 이유를 높이는 JW그룹의 인재상입니다.')],
 'GS칼텍스': [('신뢰', '